# Publishing

Publishing your results can mean several things:
- writing a manuscript and submitting the results to a journal for (double-blind) peer review
- creating a data publication, ideally submitted with your text manuscript, for data transparency & reproducibility
- publish your git repository, making it available for others to find and read your notebooks

We provide a short step-by-step guide on how to publish Jupyter notebooks together with the generated visuals and output as a data publication.

## Publish data

### Make a release version

When you are finished with your work, e.g. before submitting your manuscript for the first round of review, create a git release for your notebook repository and give it a version number:
```bash
git tag -a v1.0.0 -m "Major release before submitting to Journal"
git push --tags
```

```{hint}
This adds a marker to your Git repository that can be easily found and referenced at any later stage. If you submit a minor or major revision at a later date, add another version tag to describe your progress.
```

## Create HTML versions of all your notebooks

This is an optional step, but recommended because reviewers may not have Jupyter Lab to open your `*.ipynb` notebooks. By converting notebooks to HTML format, you can archive any code together with the generated visuals. You can convert notebooks directly in Jupyter with the below command:

In [1]:
!jupyter nbconvert --to html_toc \
    --output-dir=../out/ ./205_publish.ipynb \
    --template=../nbconvert.tpl \
    --ExtractOutputPreprocessor.enabled=False

[NbConvertApp] Converting notebook ./205_publish.ipynb to html_toc
[NbConvertApp] Writing 284729 bytes to ../out/205_publish.html


```{admonition} Add conversion command at the end of every notebook
:class: hint
It is a good idea to add this command to every notebook, so it is run after every notebook change.
```

### Create a zip file with all your output data

When you have exported all notebook HTMLs and Figures, create a ZIP file that includes all your data, notebooks, HTMLs and figures. Again, you can directly create this ZIP file in Jupyter, based on the latest git-version that we created above.

In [4]:
!cd .. && git config --system --add safe.directory '*' \
    && RELEASE_VERSION=$(git describe --tags --abbrev=0) \
    && 7z a -tzip -mx=9 out/release_$RELEASE_VERSION.zip \
    py/* out/* resources/* notebooks/*.ipynb \
    CHANGELOG.md README.md jupytext.toml nbconvert.tpl conf.json pyproject.toml \
    -x!py/__pycache__ -x!py/modules/__pycache__ -x!py/modules/.ipynb_checkpoints \
    -y > /dev/null


jupytext.toml



In [5]:
!RELEASE_VERSION=$(git describe --tags --abbrev=0) && ls -alh ../out/release_$RELEASE_VERSION.zip

-rw-r--r-- 1 root 1002 56M Mar 24 09:15 ../out/release_v0.6.1.zip


```{figure} ../resources/download.png
:name: download-figure

In the Explorer on the left, right click and select 'download'.
```

### ioerDATA

With this file you are ready to upload your data to a data repository and create a DOI so that it can be properly archived, cited and referenced.

The ioerDATA is one such repository. It is available to all IOER collaborators at [https://data.fdz.ioer.de](https://data.fdz.ioer.de).

```{hint} See the ioerDATA documentation
If you are an IOER colleague, have a look at the (internal) ioerDATA documentation at [https://docs.fdz.ioer.info/documentation/ioerdata/](https://docs.fdz.ioer.info/documentation/ioerdata/).
```

Other data repositories include [Zenodo](https://zenodo.org/).

## Publishing code

In addition to a data repository, you can make your git repository available through (for example) Gitlab or Github. If you are in the middle of peer review, you may want to temporarily remove or redact any names.

```{hint} Using Github pages
You can configure Github to publish your HTML converted notebooks to Github Pages at github.io. See the [Quickstart for GitHub Pages](https://docs.github.com/en/pages/quickstart).
```

✨ Then, spread the love! 💖 Share your notebook links with others on social media 📢, in communities 🤝, and beyond! 🚀